In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time

symbols = ["H","Be","H"]
geometry = np.array([ -1.3, 0.0, 0.0,0.0, 0.0, 0.0, 1.3, 0.0, 0.0])

In [ ]:
H, qubits = qchem.molecular_hamiltonian(
    symbols,
    geometry,
    active_electrons=4,
    active_orbitals=6
)

active_electrons = 4

singles, doubles = qchem.excitations(active_electrons, qubits)

print(f"Total number of excitations = {len(singles) + len(doubles)}")

Total number of excitations = 92


In [ ]:
hf_state = qchem.hf_state(active_electrons, qubits)


def circuit_1(params, excitations):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)
    return qml.expval(H)

In [ ]:
dev = qml.device("default.qubit", wires=qubits)
cost_fn = qml.QNode(circuit_1, dev)

circuit_gradient = qml.grad(cost_fn, argnum=0)

params = [0.0] * len(doubles)
grads = circuit_gradient(params, excitations=doubles)

for i in range(len(doubles)):
    print(f"Excitation : {doubles[i]}, Gradient: {grads[i]}")

Excitation : [0, 1, 4, 5], Gradient: -0.0206386612660292
Excitation : [0, 1, 4, 7], Gradient: 0.0
Excitation : [0, 1, 4, 9], Gradient: 0.0
Excitation : [0, 1, 4, 11], Gradient: 0.0
Excitation : [0, 1, 5, 6], Gradient: 0.0
Excitation : [0, 1, 5, 8], Gradient: 0.0
Excitation : [0, 1, 5, 10], Gradient: 0.0
Excitation : [0, 1, 6, 7], Gradient: -0.020638661266030648
Excitation : [0, 1, 6, 9], Gradient: 0.0
Excitation : [0, 1, 6, 11], Gradient: 0.0
Excitation : [0, 1, 7, 8], Gradient: 0.0
Excitation : [0, 1, 7, 10], Gradient: 0.0
Excitation : [0, 1, 8, 9], Gradient: -0.10141705820985372
Excitation : [0, 1, 8, 11], Gradient: 0.0
Excitation : [0, 1, 9, 10], Gradient: 0.0
Excitation : [0, 1, 10, 11], Gradient: -0.08163119957412635
Excitation : [0, 2, 4, 6], Gradient: 0.0
Excitation : [0, 2, 4, 8], Gradient: 0.0
Excitation : [0, 2, 4, 10], Gradient: 0.0
Excitation : [0, 2, 6, 8], Gradient: 0.0
Excitation : [0, 2, 6, 10], Gradient: 0.0
Excitation : [0, 2, 8, 10], Gradient: 0.004556396761732119
Ex

In [ ]:
doubles_select = [doubles[i] for i in range(len(doubles)) if abs(grads[i]) > 1.0e-5]
doubles_select

[[0, 1, 4, 5],
 [0, 1, 6, 7],
 [0, 1, 8, 9],
 [0, 1, 10, 11],
 [0, 2, 8, 10],
 [0, 3, 8, 11],
 [0, 3, 9, 10],
 [1, 2, 8, 11],
 [1, 2, 9, 10],
 [1, 3, 9, 11],
 [2, 3, 4, 5],
 [2, 3, 6, 7],
 [2, 3, 8, 9],
 [2, 3, 10, 11]]

In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=0.5)

params_doubles = np.zeros(len(doubles_select), requires_grad=True)

for n in range(20):
    params_doubles = opt.step(cost_fn, params_doubles, excitations=doubles_select)

In [ ]:
def circuit_2(params, excitations, gates_select, params_select):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, gate in enumerate(gates_select):
        if len(gate) == 4:
            qml.DoubleExcitation(params_select[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params_select[i], wires=gate)

    for i, gate in enumerate(excitations):
        if len(gate) == 4:
            qml.DoubleExcitation(params[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params[i], wires=gate)
    return qml.expval(H)

In [ ]:
cost_fn = qml.QNode(circuit_2, dev)
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(singles)

grads = circuit_gradient(
    params,
    excitations=singles,
    gates_select=doubles_select,
    params_select=params_doubles
)

for i in range(len(singles)):
    print(f"Excitation : {singles[i]}, Gradient: {grads[i]}")

Excitation : [0, 4], Gradient: 0.0
Excitation : [0, 6], Gradient: 0.0
Excitation : [0, 8], Gradient: 0.0
Excitation : [0, 10], Gradient: 0.005543526715525948
Excitation : [1, 5], Gradient: 0.0
Excitation : [1, 7], Gradient: 0.0
Excitation : [1, 9], Gradient: 0.0
Excitation : [1, 11], Gradient: -0.005543526989549823
Excitation : [2, 4], Gradient: 0.0
Excitation : [2, 6], Gradient: 0.0
Excitation : [2, 8], Gradient: 0.0086222030057358
Excitation : [2, 10], Gradient: 0.0
Excitation : [3, 5], Gradient: 0.0
Excitation : [3, 7], Gradient: 0.0
Excitation : [3, 9], Gradient: -0.008622203274572896
Excitation : [3, 11], Gradient: 0.0


In [ ]:
singles_select = [singles[i] for i in range(len(singles)) if abs(grads[i]) > 1.0e-5 ]
singles_select

[[0, 10], [1, 11], [2, 8], [3, 9]]

In [ ]:
cost_fn = qml.QNode(circuit_1, dev)

params = np.zeros(len(doubles_select + singles_select), requires_grad=True)

gates_select = doubles_select + singles_select

for n in range(20):
    t1 = time.time()
    params, energy = opt.step_and_cost(cost_fn, params, excitations=gates_select)
    t2 = time.time()
    print("n = {:},  E = {:.8f} H, t = {:.2f} s".format(n, energy, t2 - t1))

n = 0,  E = -14.79471419 H, t = 5.89 s
n = 1,  E = -14.81367955 H, t = 8.70 s
n = 2,  E = -14.81889393 H, t = 3.20 s
n = 3,  E = -14.82117026 H, t = 9.00 s
n = 4,  E = -14.82236977 H, t = 3.50 s
n = 5,  E = -14.82307415 H, t = 3.10 s
n = 6,  E = -14.82351465 H, t = 3.20 s
n = 7,  E = -14.82380014 H, t = 10.80 s
n = 8,  E = -14.82398910 H, t = 3.10 s
n = 9,  E = -14.82411586 H, t = 9.40 s
n = 10,  E = -14.82420171 H, t = 3.20 s
n = 11,  E = -14.82426026 H, t = 3.60 s
n = 12,  E = -14.82430044 H, t = 3.10 s
n = 13,  E = -14.82432814 H, t = 10.60 s
n = 14,  E = -14.82434732 H, t = 3.71 s
n = 15,  E = -14.82436064 H, t = 8.39 s
n = 16,  E = -14.82436992 H, t = 4.61 s
n = 17,  E = -14.82437640 H, t = 3.10 s
n = 18,  E = -14.82438094 H, t = 3.10 s
n = 19,  E = -14.82438412 H, t = 8.29 s
